### Mapas coropléticos (*Choropleth Maps*) - Mapa coroplético de Galicia

Pinta diferentes mapas coropléticos de Galicia

1. Un mapa coroplético segundo as tamaños/áreas dos concellos
2. Un mapa coroplético en función da poboación de cada concello

**Notas**:

1. Utiliza o Shapefile de Concellos de Galicia que utilizamos noutras ocasións
2. Calcula as áreas coas funcións xeométricas dos geodataframes
3. Descarga os datos de poboación desde o INE (..y que la fuerza te acompañe.)


**INE**: podes descargar os datos de poboación do INE, po exemplo, aquí: https://ine.es/dyngs/INEbase/operacion.htm?c=Estadistica_C&cid=1254736177011&menu=resultados&secc=1254736195458&idp=1254734710990

**Wikipedia**: tamén os podes scrappear da Wikipedia: https://gl.wikipedia.org/wiki/Concellos_de_Galicia

Ollo! porque terás que cruzar os datos do Shapefile cos datos de poboación

1. Mapa das áreas dos concellos de Galicia

In [ ]:
import pandas as pd
import geopandas as gpd
concellos = gpd.read_file('../../datasets/Concellos/Concellos_IGN.shp')
concellos.head(3)

In [ ]:
# Comprobamos o CRS
concellos.crs

In [ ]:
# Temos unha columna coa área, mais poderíamos calculala a partir da xeometría
concellos['area_km2'] = concellos.area / 10**6
concellos.head(3)

In [ ]:
# Pode ser interesante investigar a distribución da variable que queremos representar

#Mostramos a distribución de "area_km2"
import seaborn as sns
sns.histplot(concellos['area_km2'])

In [ ]:
# Calculamos os valores mínimos e máximos da variable para establecer o rango para as cores
concellos.area_km2.apply(['min','max'])

In [ ]:
import matplotlib.pyplot as plt

variable = 'area_km2'
vmin, vmax = 0, 500
colormap = 'Blues'

fig, ax = plt.subplots(1, figsize=(30, 30))
ax.axis('off')
ax.set_title('Área dos concellos de Galicia', fontdict={'fontsize': '25', 'fontweight' : '3'})

# Engadir unha nota ao pé 
ax.annotate('Fonte: Cartografía de Galicia - https://www.sergas.es/Saude-publica/GIS-Cartografia-Galicia-formato-vectorial-SHP',xy=(0.2, .05),  \
            xycoords='figure fraction', fontsize=12, \
            color='#555555')

# Lenda: barra
sm = plt.cm.ScalarMappable(cmap=colormap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm.set_array([])
fig.colorbar(sm, orientation="horizontal", fraction=0.036, pad=0.1, aspect = 30)

# Pintar a gráfica
concellos.plot(column=variable, cmap=colormap, linewidth=0.8, ax=ax, edgecolor='0.8')

# Engadir etiquetas cos nomes das provincias
# Primeiro extrae un punto representativo de cada polígono

concellos['coords'] = concellos['geometry'].apply(lambda x: x.representative_point().coords[:])
concellos['coords'] = [coords[0] for coords in concellos['coords']]

# Itera as liñas do dataset e vai engadindo as etiquetas co nome da provincia
for idx, row in concellos.iterrows():
    plt.annotate(row['Concello'], xy=row['coords'],horizontalalignment='center')

2. Mapa das poboacións dos concellos galegos

Escolleuse a opción de descargar os datos do INE, xa que cada concello ten un código que podemos relacionar cos shapefiles publicados pola Xunta, o que nos facilitará cruzar dataframes.

Temos que descargar os ficheiros por provincia: Cifras oficiales de población de los municipios españoles: Revisión del Padrón Municipal > Detalle municipal

In [ ]:
from pyaxis import pyaxis 

# Instalar antes pyaxis
#  activar contorno: p.ex: conda activate nome_contorno
# instalar paquete con pip: pip install pyaxis

In [ ]:
px = pyaxis.parse('../../datasets/Concellos/poboacion/poboacion_acorunha.px', encoding='ISO-8859-2')

In [ ]:
px

In [ ]:
type(px)

In [ ]:
px['DATA']

In [ ]:
type(px['DATA'])

In [ ]:
px_acorunha = pyaxis.parse('../../datasets/Concellos/poboacion/poboacion_acorunha.px', encoding='ISO-8859-2')['DATA']
px_acorunha

In [ ]:
px_lugo = pyaxis.parse('../../datasets/Concellos/poboacion/poboacion_lugo.px', encoding='ISO-8859-2')['DATA']
px_ourense = pyaxis.parse('../../datasets/Concellos/poboacion/poboacion_ourense.px', encoding='ISO-8859-2')['DATA']
px_pontevedra = pyaxis.parse('../../datasets/Concellos/poboacion/poboacion_pontevedra.px', encoding='ISO-8859-2')['DATA']
poboacion = pd.concat([px_acorunha,px_lugo,px_ourense,px_pontevedra])
poboacion

In [ ]:
poboacion['codigo'] = poboacion.Municipios.apply(lambda x:x.split()[0])
poboacion.head()

In [ ]:
poboacion['nomeConcello'] = poboacion.Municipios.apply(lambda x:x.split(' ',1)[1])
poboacion.head()

In [ ]:
poboacion.drop(columns=['Municipios','Sexo'],inplace=True)
poboacion

In [ ]:
poboacion.rename(columns={'DATA':'poboacion'},inplace=True)

In [ ]:
poboacion.info()

In [ ]:
poboacion['poboacion'] = poboacion.poboacion.astype(float)

In [ ]:
concellos_con_poboacion = concellos.merge(poboacion, how='left', left_on='CODIGOINE', right_on='codigo')
concellos_con_poboacion

In [ ]:
# Calculamos os valores mínimos e máximos da variable para establecer o rango para as cores
concellos_con_poboacion.poboacion.dropna().apply(['min','max'])

In [ ]:
import matplotlib.pyplot as plt

variable = 'poboacion'
vmin, vmax = 0, 30000
colormap = 'Blues'

fig, ax = plt.subplots(1, figsize=(30, 30))
ax.axis('off')
ax.set_title('Poboación dos concellos de Galicia', fontdict={'fontsize': '35', 'fontweight' : '5'})

# Engadir unha nota ao pé 
ax.annotate('Fonte: Cartografía de Galicia - https://www.sergas.es/Saude-publica/GIS-Cartografia-Galicia-formato-vectorial-SHP',xy=(0.2, .05),  \
            xycoords='figure fraction', fontsize=12, \
            color='#555555')

# Lenda: barra
sm = plt.cm.ScalarMappable(cmap=colormap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm.set_array([])
fig.colorbar(sm, orientation="horizontal", fraction=0.036, pad=0.1, aspect = 30)

# Pintar a gráfica
concellos_con_poboacion.plot(column=variable, cmap=colormap, linewidth=0.8, ax=ax, edgecolor='0.8')

# Engadir etiquetas cos nomes das provincias
# Primeiro extrae un punto representativo de cada polígono

concellos['coords'] = concellos['geometry'].apply(lambda x: x.representative_point().coords[:])
concellos['coords'] = [coords[0] for coords in concellos['coords']]

# Itera as liñas do dataset e vai engadindo as etiquetas co nome da provincia
for idx, row in concellos.iterrows():
    plt.annotate(row['Concello'], xy=row['coords'],horizontalalignment='center')

In [ ]:
fig.savefig('concellos_poboacion.png',facecolor='white')

In [ ]:
# Tamén sería interesante un mapa coa densidade de poboación

In [ ]:
concellos_con_poboacion['densidade'] = concellos_con_poboacion.poboacion / concellos_con_poboacion.area_km2
concellos_con_poboacion.head(3)

In [ ]:
# Calculamos os valores mínimos e máximos da variable para establecer o rango para as cores
concellos_con_poboacion.densidade.dropna().apply(['min','max'])

In [ ]:
sns.boxplot(concellos_con_poboacion.densidade)

In [ ]:
import matplotlib.pyplot as plt

variable = 'densidade'
vmin, vmax = 0, 6500
colormap = 'Purples'

fig, ax = plt.subplots(1, figsize=(30, 30))
ax.axis('off')
ax.set_title('Densidade de poboación nos concellos de Galicia', fontdict={'fontsize': '35', 'fontweight' : '5'})

# Engadir unha nota ao pé 
ax.annotate('Fonte: Cartografía de Galicia - https://www.sergas.es/Saude-publica/GIS-Cartografia-Galicia-formato-vectorial-SHP',xy=(0.2, .05),  \
            xycoords='figure fraction', fontsize=12, \
            color='#555555')

# Lenda: barra
sm = plt.cm.ScalarMappable(cmap=colormap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm.set_array([])
fig.colorbar(sm, orientation="horizontal", fraction=0.036, pad=0.1, aspect = 30)

# Pintar a gráfica
concellos_con_poboacion.plot(column=variable, cmap=colormap, linewidth=0.8, ax=ax, edgecolor='0.8')

# Engadir etiquetas cos nomes das provincias
# Primeiro extrae un punto representativo de cada polígono

concellos['coords'] = concellos['geometry'].apply(lambda x: x.representative_point().coords[:])
concellos['coords'] = [coords[0] for coords in concellos['coords']]

# Itera as liñas do dataset e vai engadindo as etiquetas co nome da provincia
for idx, row in concellos.iterrows():
    plt.annotate(row['Concello'], xy=row['coords'],horizontalalignment='center')

In [ ]:
fig.savefig('concellos_densidade_poboacion.png',facecolor='white')